In [1]:
# Import dependencies
import time
import numpy as np
import pandas as pd
from scipy.stats import norm
from datetime import datetime
import matplotlib.pyplot as plt

In [13]:
# Parameters
S0 = 83.5900  # Spot price
K = 84.1080   # Strike price
T = 0.5       # Time to expiry in years (6 months)
rd = 0.0513   # Domestic risk-free rate
rf = 0.0645   # Foreign risk-free rate
vol = (0.02527 + 0.03102) / 2  # Average implied volatility
N = 126  # Number of time steps (approximately 6 months of trading days)
M = 10000  # Number of simulation paths

# Time to start and end of averaging period
T1 = 0.0  # Starting time (now)
T2 = T    # Ending time (expiry)

T_tT2 = T2
T_tT1 = T1
print("Start averaging from T1", round(T_tT1, 2), "to T2", round(T_tT2, 2))


Start averaging from T1 0.0 to T2 0.5


In [9]:

def blackScholes(rd,rf, S, K, T, sigma, type="c"):
    "Calculate BS price of call/put"
    d1 = (np.log(S/K) + (rd - rf + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    try:
        if type == "c":
            price = S*np.exp(-rf*T)*norm.cdf(d1, 0, 1) - K*np.exp(-rd*T)*norm.cdf(d2, 0, 1)
        elif type == "p":
            price = K*np.exp(-rd*T)*norm.cdf(-d2, 0, 1) - S*np.exp(-rf*T)*norm.cdf(-d1, 0, 1)
        return price
    except:
        print("Please confirm option type, either 'c' for Call or 'p' for Put!")


print("Black Scholes Option Price: ", round(blackScholes(rd,rf, S0, K, T, vol, "c"),2)*100/S0)

Black Scholes Option Price:  0.7895681301591099


In [15]:
obs_times = np.linspace(T_tT1, T_tT2, N+1)
# Include starting time, uneven time deltas
obs_times[0] = 0
dt = np.diff(obs_times)
print("Number of time steps:", len(dt))

start_time = time.time()

nudt = np.full(shape=(N, M), fill_value=0.0)
volsdt = np.full(shape=(N, M), fill_value=0.0)

# Precompute constants
for i in range(N):
    nudt[i,:] = (rd - rf - 0.5 * vol**2) * dt[i]
    volsdt[i,:] = vol * np.sqrt(dt[i])


# Monte Carlo Method
Z = np.random.normal(size=(N, M))
delta_St = nudt + volsdt * Z
ST = S0 * np.cumprod(np.exp(delta_St), axis=0)
AT = np.cumsum(ST, axis=0) / N
ST = np.concatenate((np.full(shape=(1, M), fill_value=S0), ST))

CT = np.maximum(0, AT[-1] - K)
C0_fast = np.exp(-rd * T) * np.sum(CT) / M

sigma = np.sqrt(np.sum((np.exp(-rd * T) * CT - C0_fast)**2) / (M - 1))
sigma = np.std(np.exp(-rd * T) * CT)
SE_fast = sigma / np.sqrt(M)

time_comp_fast = round(time.time() - start_time, 4)
print("Call value is {0} with SE +/- {1}".format(np.round(C0_fast, 3), np.round(SE_fast, 3)))
print("Computation time is: ", time_comp_fast)


Number of time steps: 126
Call value is 0.107 with SE +/- 0.003
Computation time is:  0.0911


In [11]:
# fast steps - discretized every day (288 trading intervals)
N = Q123_days

obs_times = np.linspace(T_tT1,T_tT2,N+1)
# Include starting time, uneven time delta's
obs_times[0]=0
dt = np.diff(obs_times)
print("Number of time steps:", len(dt))

Number of time steps: 181


In [12]:
# slow steps - discretized every day
N = Q123_days

T_tT2 = ((T2-t).days+1)/365
T_tT1 = (T1-t).days/365
print("Start averaging from T1", round(T_tT1,2), "to T2", round(T_tT2,2))

obs_times = np.linspace(T_tT1,T_tT2,N+1)
# Include starting time, uneven time delta's
obs_times[0]=0
dt = np.diff(obs_times)
print("Number of time steps:", len(dt))

start_time = time.time()

nudt = np.full(shape=(N,M), fill_value=0.0)
volsdt = np.full(shape=(N,M), fill_value=0.0)

# Precompute constants
for i in range(N):
    nudt[i,:] = (r - 0.5*vol**2)*dt[i]
    volsdt[i,:] = vol*np.sqrt(dt[i])

# Monte Carlo Method
Z = np.random.normal(size=(N, M))
delta_St = nudt + volsdt*Z
ST = S0*np.cumprod( np.exp(delta_St), axis=0)
AT = np.cumsum(ST, axis=0)/N
ST = np.concatenate( (np.full(shape=(1, M), fill_value=S0), ST ) )

CT = np.maximum(0, AT[-1] - K)
C0_fast = np.exp(-r*T)*np.sum(CT)/M

sigma = np.sqrt( np.sum( (np.exp(-r*T)*CT - C0_fast)**2) / (M-1) )
sigma = np.std(np.exp(-r*T)*CT)
SE_fast = sigma/np.sqrt(M)

time_comp_fast = round(time.time() - start_time,4)
print("Call value is ${0} with SE +/- {1}".format(np.round(C0_fast,3),np.round(SE_fast,3)))
print("Computation time is: ", time_comp_fast)

Call value is $0.273 with SE +/- 0.015
Computation time is:  0.0117


In [17]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters from the image
S0 = 83.5900  # Spot price
K = 84.1080   # Strike price
expiry = 0.5  # 6 months in years
rd = 0.0513   # Domestic risk-free rate (USD SOFR)
rf = 0.0645   # Foreign risk-free rate (INR Depo)
vol = (0.02527 + 0.03102) / 2  # Average implied volatility
N = 126  # Number of time steps (approximately 6 months of trading days)
M = 10000  # Number of simulation paths

# Time to start and end of averaging period
T1 = 0.0  # Starting time (now)
T2 = expiry  # Ending time (expiry)

T_tT2 = T2
T_tT1 = T1
print("Start averaging from T1", round(T_tT1, 2), "to T2", round(T_tT2, 2))

obs_times = np.linspace(T_tT1, T_tT2, N+1)
# Include starting time, uneven time deltas
obs_times[0] = 0
dt = np.diff(obs_times)
print("Number of time steps:", len(dt))

start_time = time.time()

nudt = np.full(shape=(N, M), fill_value=0.0)
volsdt = np.full(shape=(N, M), fill_value=0.0)

# Precompute constants
for i in range(N):
    nudt[i,:] = (rd - rf - 0.5 * vol**2) * dt[i]
    volsdt[i,:] = vol * np.sqrt(dt[i])

# Monte Carlo Method
Z = np.random.normal(size=(N, M))
delta_St = nudt + volsdt * Z
ST = S0 * np.cumprod(np.exp(delta_St), axis=0)
AT = np.cumsum(ST, axis=0) / N
ST = np.concatenate((np.full(shape=(1, M), fill_value=S0), ST))

CT = np.maximum(0, AT[-1] - K)
C0_fast = np.exp(-rd * T) * np.sum(CT) / M

sigma = np.sqrt(np.sum((np.exp(-rd * T) * CT - C0_fast)**2) / (M - 1))
sigma = np.std(np.exp(-rd * T) * CT)
SE_fast = sigma / np.sqrt(M)

time_comp_fast = round(time.time() - start_time, 4)
print("Call value is ${0} with SE +/- {1}".format(np.round(C0_fast, 3), np.round(SE_fast, 3)))
print("Computation time is: ", time_comp_fast)

# Black-Scholes model for lookback option comparison
dt_bs = T / N  # Time step size
nudt_bs = (rd - rf - 0.5 * vol ** 2) * dt_bs
volsdt_bs = vol * np.sqrt(dt_bs)

# Simulate paths using Black-Scholes dynamics
np.random.seed(0)
Z_bs = np.random.normal(size=(N, M))
S_paths_bs = np.zeros((N+1, M))
S_paths_bs[0] = S0

for i in range(1, N+1):
    S_paths_bs[i] = S_paths_bs[i-1] * np.exp(nudt_bs + volsdt_bs * Z_bs[i-1])

# Calculate maximum prices for each path (Lookback feature)
S_max_bs = np.max(S_paths_bs, axis=0)

# Lookback call option payoff: max(S_max - S0, 0)
payoffs_bs = np.maximum(S_max_bs - S0, 0)
lookback_price_bs = np.exp(-rd * T) * np.mean(payoffs_bs)

print(f"The simulated lookback call option price using Black-Scholes model is: {lookback_price_bs:.2f}")


Start averaging from T1 0.0 to T2 0.5
Number of time steps: 126
Call value is $0.107 with SE +/- 0.003
Computation time is:  0.0884
The simulated lookback call option price using Black-Scholes model is: 0.97


In [21]:
import numpy as np
import datetime
import time

# Given data for lookback call option
S0 = 83.59          # Spot exchange rate
K = 84.108          # Strike price
vol = 0.028145      # Volatility
rd = 0.06446        # Domestic risk-free rate
rf = 0.05134        # Foreign risk-free rate
T = ((datetime.date(2025,1,17) - datetime.date(2024,7,17)).days) / 365  # Time in years

# Assumed parameters for the Heston model
kappa = 2.0         # Mean-reversion rate
theta = 0.02        # Long-term mean variance
sigma = 0.2         # Volatility of volatility
vt0 = vol**2        # Initial variance
rho = -0.5          # Correlation

N = 52      # number of time intervals
M = 1000    # number of simulations

# Start timing
start_time = time.time()

# Precompute constants
dt = T / N
kappadt = kappa * dt
sigmasdt = sigma * np.sqrt(dt)

# Standard Error Placeholders
sum_CT = 0
sum_CT2 = 0

# Monte Carlo Method
for i in range(M):

    St = S0
    vt = vt0
    St_max = S0

    for j in range(N):

        # Generate Wiener variables
        W = np.random.normal(0.0, 1.0, size=(2))
        
        # Correlated Brownian motions
        W1 = W[0]
        W2 = rho * W1 + np.sqrt(1 - rho**2) * W[1]

        # Variance process
        vt = vt + kappadt * (theta - vt) + sigmasdt * np.sqrt(vt) * W1

        # Asset process
        St = St * np.exp((rd - rf - 0.5 * vt) * dt + np.sqrt(vt * dt) * W2)

        if St > St_max:
            St_max = St

    CT = max(0, St_max - K)
    sum_CT += CT
    sum_CT2 += CT * CT

# Compute Expectation and SE
C0_slow = np.exp(-rd * T) * sum_CT / M
SE_slow = np.sqrt((sum_CT2 - sum_CT * sum_CT / M) * np.exp(-2 * rd * T) / (M - 1)) / np.sqrt(M)

# End timing
time_comp_slow = round(time.time() - start_time, 4)

print("Lookback Call value is ${0} with SE +/- {1}".format(np.round(C0_slow, 3), np.round(SE_slow, 3)))
print("Computation time is:", time_comp_slow)


C:\Users\91751\AppData\Local\Temp\ipykernel_26744\2536154765.py:55: RuntimeWarning: invalid value encountered in sqrt
  St = St * np.exp((rd - rf - 0.5 * vt) * dt + np.sqrt(vt * dt) * W2)
C:\Users\91751\AppData\Local\Temp\ipykernel_26744\2536154765.py:52: RuntimeWarning: invalid value encountered in sqrt
  vt = vt + kappadt * (theta - vt) + sigmasdt * np.sqrt(vt) * W1


Lookback Call value is $1.864 with SE +/- 0.07
Computation time is: 0.3395


In [27]:
import numpy as np
import datetime
import time

# Given data for lookback call option
S0 = 83.59          # Spot exchange rate
K = 84.108          # Strike price
vol = 0.028145      # Volatility
rd = 0.06446        # Domestic risk-free rate
rf = 0.05134        # Foreign risk-free rate
T = ((datetime.date(2025,1,17) - datetime.date(2024,7,17)).days) / 365  # Time in years

# Assumed parameters for the Heston model
kappa = 2.0         # Mean-reversion rate
theta = 0.02        # Long-term mean variance
sigma = 0.2         # Volatility of volatility
vt0 = vol**2        # Initial variance
rho = -0.5          # Correlation

N = 52      # number of time intervals
M = 1000    # number of simulations

# Start Timer
start_time = time.time()

# Precompute constants
dt = T / N

# Heston model adjustments for time steps
kappadt = kappa * dt
sigmasdt = sigma * np.sqrt(dt)

# Generate Wiener variables
W = np.random.normal(0.0, 1.0, size=(N, M, 2))

# Arrays for storing prices and variances
St = np.full(shape=(N + 1, M), fill_value=S0)
vt = np.full(shape=(N + 1, M), fill_value=vt0)

# Array for storing maximums
St_max = np.full(shape=(M), fill_value=S0)

for j in range(1, N + 1):

    # Simulate variance processes
    vt[j] = vt[j - 1] + kappadt * (theta - vt[j - 1]) + sigmasdt * np.sqrt(vt[j - 1]) * W[j - 1, :, 0]

    # Simulate log asset prices
    nudt = (rd - rf - 0.5 * vt[j]) * dt
    St[j] = St[j - 1] * np.exp(nudt + np.sqrt(vt[j] * dt) * W[j - 1, :, 1])

    mask = np.where(St[j] > St_max)
    St_max[mask] = St[j][mask]

# Compute Expectation and SE
CT = np.maximum(0, St_max - K)
C0_fast = np.exp(-rd * T) * np.sum(CT) / M

SE_fast = np.sqrt(np.sum((np.exp(-rd * T) * CT - C0_fast)**2) / (M - 1)) / np.sqrt(M)

time_comp_fast = round(time.time() - start_time, 4)
print("Lookback Call value is ${0} with SE +/- {1}".format(np.round(C0_fast, 2), np.round(SE_fast, 2)))
print("Computation time is:", time_comp_fast)


Lookback Call value is $3.16 with SE +/- 0.11
Computation time is: 0.0061


C:\Users\91751\AppData\Local\Temp\ipykernel_26744\678758036.py:50: RuntimeWarning: invalid value encountered in sqrt
  St[j] = St[j - 1] * np.exp(nudt + np.sqrt(vt[j] * dt) * W[j - 1, :, 1])
C:\Users\91751\AppData\Local\Temp\ipykernel_26744\678758036.py:46: RuntimeWarning: invalid value encountered in sqrt
  vt[j] = vt[j - 1] + kappadt * (theta - vt[j - 1]) + sigmasdt * np.sqrt(vt[j - 1]) * W[j - 1, :, 0]
